In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
contests = pd.read_csv('data/ContestsWithoutCheats.csv').set_index('ContestId')
contests.tail(1)

,Success,SportName_NFL,SportName_PGA,SportName_SOCC,SportName_MLB,SportName_NAS,SportName_CFB,SportName_MMA,SportName_LOL,SportName_NHL,...,40_HoursOut,41_HoursOut,42_HoursOut,43_HoursOut,44_HoursOut,45_HoursOut,46_HoursOut,47_HoursOut,48_HoursOut,RestOf_HoursOut
ContestId,,,,,,,,,,,,,,,,,,,,,
58300706,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Required Accuracy:

In [6]:
numFail = len(contests.loc[contests['Success']==0])
print('Percent of contests that breakeven:', 1-numFail/len(contests))

Percent of contests that breakeven: 0.9557599602529844


In [7]:
CLF_contests = contests.loc[:].dropna()

In [14]:
CLF_Features = CLF_contests.columns.drop('Success')

In [10]:
CLF_contests['is_train'] = np.random.uniform(0, 1, len(CLF_contests)) <= .75
train, test = CLF_contests[CLF_contests['is_train']==True], CLF_contests[CLF_contests['is_train']==False]
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 279166
Number of observations in the test data: 93085


In [11]:
actual = test['Success']
y = train['Success']

In [15]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(train[CLF_Features], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [16]:
preds = clf.predict(test[CLF_Features])
trueIds = test[['Success']]

In [17]:
featureImportanceDF = pd.DataFrame(data={"Feature":CLF_Features, "Importance":clf.feature_importances_})
featureImportanceDF = featureImportanceDF.sort_values(by=['Importance'],ascending=False)

In [18]:
# featureImportanceDF.head(50)
featureImportanceDF.to_csv("data/RawForestFeatures.csv")

In [24]:
featureImportanceDF.head(50)

,Feature,Importance
91,MaxNumberPlayers_Scaled,0.059039
96,TopPrize_Scaled,0.059033
90,TotalPrizeAmount_Scaled,0.058044
98,SuccessThreshold_Scaled,0.056359
89,EntryFeeAmount_Scaled,0.056043
95,PaidUsersInDraftGroup_Scaled,0.052016
97,MaxPayoutPosition_Scaled,0.046893
94,DraftablePlayersInSet_Scaled,0.032075
76,Payout_Minute_Y,0.024365
75,Payout_Minute_X,0.023592


In [25]:
accuracy_score(actual, preds)

0.9590911532470323

In [19]:
confuse = pd.DataFrame({'Predict':preds, 'Real':actual})

In [20]:
successes =  len(test.loc[test['Success']==1.0])
fails = len(test.loc[test['Success']==0.0])

In [21]:
print("Contest Reach Threshold? fails:", fails, "---", "successes:", successes)

Contest Reach Threshold? fails: 4091 --- successes: 3973


In [22]:
print(confusion_matrix(actual, preds))

[[3481  610]
 [1162 2811]]


In [23]:
# true.columns = ['RealVal']
trueDF = pd.DataFrame(data=actual)
trueDF.columns = ['RealVal']
trueDF = trueDF.assign(PredVal= preds.tolist())


In [24]:
failDF = trueDF.loc[trueDF['RealVal'] != trueDF['PredVal']]
falsePositives = failDF.loc[failDF['PredVal']==1][[]]
falseNegatives = failDF.loc[failDF['PredVal']==0][[]]
falsePositivesDF = pd.merge(falsePositives, rawContests, on='ContestId', how='left')
falseNegativesDF = pd.merge(falseNegatives, rawContests, on='ContestId', how='left')

NameError: name 'rawContests' is not defined

In [ ]:
# falsePositivesDF.to_csv("data/ForJon/RawFalsePositives.csv")
# falseNegativesDF.to_csv("data/ForJon/RawFalseNegatives.csv")